In [1]:
%gui qt

In [2]:
from fimpy.pipeline.alignment import align_volumes_with_filtering
from fimpy.pipeline.general import downsample, make_anatomy
from fimpy.pipeline.source_extraction import correlation_map, grow_rois, extract_traces
from split_dataset import SplitDataset
import napari
import flammkuchen as fl
import numpy as np
from pathlib import Path
from tqdm import tqdm

from fimpy.pipeline.general import dff
from scipy.ndimage import zoom

ModuleNotFoundError: No module named 'fimpy.pipeline.source_extraction'

In [ ]:
array_baseline = SplitDataset(Path(r"\\FUNES\Shared\experiments\virtual_gradients_experiments\Batch_1\210504_f1\imaging\temp\downsampled"))[400:480,:,:,:].mean(0)

In [16]:
mask = fl.load((Path(r"\\FUNES\Shared\experiments\virtual_gradients_experiments\Batch_1\210504_f1\imaging\temp\brain_mask.h5")))

In [28]:
down_mask = zoom(mask,(1,1/4,1/4))

In [16]:
master_path = Path(r"\\FUNES\Shared\Ema\data_scc")

In [15]:
list_exp = [Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\7112020_f1_ok\7112020_f1_ok"),
            Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\7112020_f0_piunochesi_ok\7112020_f0_piunochesi"),
            Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\6112020_f2_ok\6112020_f2_ok")]
#list(master_path.glob("*3"))

In [17]:
list_exp = [Path(r"\\FUNES\Shared\Ruben\from MPIN\J-E0041_temperature\ls_1120\5112020_f1_ok\5112020_f1_ok")]


In [20]:
n = 0

In [21]:
sd = SplitDataset(list_exp[n] / "original")
contrast_limits = (np.percentile(sd[0, :, :, :], 1), np.percentile(sd[0, :, :, :], 99.99))
v = napari.view_image(sd, name="Data", contrast_limits=contrast_limits, multiscale=False,blending="additive")


### Align

In [18]:
%%time
for exp_path in tqdm(list_exp):
    align_volumes_with_filtering(SplitDataset(exp_path / "original"),
                                        output_dir=None,
                                        ref_wnd_hs=25,
                                        fft_reference=None,
                                        register_every=1,
                                        reg_halfwin=1,
                                        prefilter_sigma=3.3,
                                        block_size=120,
                                        verbose=True,
                                        crop=False,
                                        n_jobs=1
                                    )

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

finding shifts...
Finding shifts at  0
Finding shifts at  1
Finding shifts at  2
Finding shifts at  3
Finding shifts at  4
Finding shifts at  5
Finding shifts at  6
Finding shifts at  7
Finding shifts at  8
Finding shifts at  9
Finding shifts at  10
Finding shifts at  11
Finding shifts at  12
Finding shifts at  13
Finding shifts at  14
Finding shifts at  15
Finding shifts at  16
Finding shifts at  17
Finding shifts at  18
Finding shifts at  19
Finding shifts at  20
Finding shifts at  21
Finding shifts at  22
Finding shifts at  23
Finding shifts at  24
Finding shifts at  25
Finding shifts at  26
Finding shifts at  27
Finding shifts at  28
Finding shifts at  29
Finding shifts at  30
Finding shifts at  31
Finding shifts at  32
Finding shifts at  33
Finding shifts at  34
Finding shifts at  35
Finding shifts at  36
Finding shifts at  37
Finding shifts at  38
Finding shifts at  39
Finding shifts at  40
Finding shifts at  41
Finding shifts at  42
Finding shifts at  43
Finding shifts at  44
Fi

100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [36:41:30<00:00, 132090.00s/it]


Wall time: 1d 12h 41min 30s


In [28]:
n = 1

In [29]:
sd_post = SplitDataset(list_exp[n] / "aligned")
contrast_limits = (np.percentile(sd_post[0, :, :, :], 50), np.percentile(sd_post[0, :, :, :], 99.99))
sd_pre = SplitDataset(list_exp[n] / "original")

In [30]:
v = napari.view_image(sd_pre, name="Pre align", contrast_limits=contrast_limits, multiscale=False,colormap="magenta",blending="additive")
v.add_image(sd_post, name="Post align", contrast_limits=contrast_limits, multiscale=False,colormap="green",blending="additive")

<Image layer 'Post align' at 0x26980240108>

### Downsample

In [7]:
%%time
for exp_path in tqdm(list_exp):
    downsample(SplitDataset(exp_path / "aligned"),downsampling=(1,1,4,4),n_jobs=2,method=np.mean)

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [46:17<00:00, 2777.92s/it]


Wall time: 46min 18s


### Anatomy

In [30]:
%%time
for exp_path in tqdm(list_exp):
    make_anatomy(SplitDataset(exp_path / "aligned"), n_jobs=1, block_size=(1, 50, 50))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Correlation Map

In [28]:
%%time
for exp_path in tqdm(list_exp):
    correlation_map(SplitDataset(exp_path / "aligned"),
                    window_size=(1, 3, 3),
                    n_jobs=1,
                   )

  0%|                                                                                            | 0/1 [00:01<?, ?it/s]


MemoryError: Unable to allocate array with shape (66000, 1, 1024, 1024) and data type uint16

In [15]:
correlation_map(SplitDataset(p / "aligned"), window_size=(1, 3, 3), n_jobs=1)

In [ ]:
n = 0

In [44]:
v = napari.view_image(SplitDataset(list_exp[0] / "correlation_map")[:,:,:], colormap="gray", blending="additive")
v.add_image(SplitDataset(list_exp[1] / "correlation_map")[:,:,:], colormap="gray", blending="additive")
v.add_image(SplitDataset(list_exp[2] / "correlation_map")[:,:,:], colormap="gray", blending="additive")

<Image layer 'Image [2]' at 0x2698a6468c8>

In [10]:
time_lims = (100, 3*60*25)
settings = dict(
    init_corr_thresh=0.09,
    max_labels=sys.maxsize,
    final_corr_threshold=0.2,
    max_radius=6,
    corr_thresh_inc_dist=3,
    max_investigate=1000,
    min_area=3,
    across_planes=False,
    voxel_size=(10.384615384615385, 0.6, 0.6))

In [12]:
path = Path(r"\\FUNES\Shared\Ema\data_bce\imaging\Fish_0\Caudal_session")

In [6]:
# grow_rois(
#     SplitDataset(path /"imaging" / "aligned"),
#     SplitDataset(path /"imaging" / "correlation_map"),
#     n_jobs=1,
#     blocksize=(1, 562, 612),
#     padding=(0, 0, 0), 
#     time_lims=time_lims, 
#     **settings)

In [1]:
# grow_rois(
#     SplitDataset(p / "aligned"),
#     SplitDataset(p / "correlation_map"),
#     n_jobs=1,
#     blocksize=(1, 562, 612),
#     padding=(0, 0, 0), 
#     time_lims=time_lims, 
#     **settings)

In [24]:
%%time
for exp_path in tqdm(list_exp):
    grow_rois(
    SplitDataset(exp_path / "aligned"),
    SplitDataset(exp_path / "correlation_map"),
    n_jobs=1,
    blocksize=(1, 562, 612),
    padding=(0, 0, 0), 
    time_lims=time_lims, 
    **settings)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


IndexError: list index out of range

In [25]:
merg_rois = fl.load(p/"merged_rois.h5")['stack_3D']

In [26]:
v = napari.view_image(SplitDataset(p / "correlation_map")[:,:,:], colormap="gray")
v.add_labels(merg_rois)

<Labels layer 'merg_rois' at 0x1543cc41748>

In [27]:
brain_mask = (v.layers[-1]._data == 0).astype(int)

In [28]:
fl.save(p/"roi_to_use.h5", merg_rois*brain_mask)

In [14]:
roi_stack = fl.load(exp_path/"merged_rois.h5")['stack_3D']

In [25]:
%%time
for exp_path in tqdm(list_exp):
#     roi_stack = fl.load(exp_path /"imaging"/ "roi_to_use.h5")
    roi_stack = fl.load(exp_path/"merged_rois.h5")['stack_3D']
    extract_traces(SplitDataset(exp_path / "aligned"), roi_stack, n_jobs=1, block_duration=10)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


OSError: ``\\FUNES\Shared\Ema\data_scc\f3\merged_rois.h5`` does not exist